In [1]:
import pickle as pk
import argparse
import pickle as pk
import time
from re import S
import torch
import copy
from tqdm import tqdm
import torch.nn as nn
from pytorch_lightning import seed_everything
from torch.nn import functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

from src.data_utils import *
#from src.model_utils import setupTokenizer
from src.datasethandler import NarrationDataSet,train_data_permutated_path,test_data_path,DataSetLoader
from src.inferenceUtils import PerformanceNarrator
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback,T5Config
from src.modeling_bart import BartNarrationModel
from src.modeling_t5 import T5NarrationModel
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from classification_performance_description_inference import ClassificationPerformanceNarration
trained_model_path = "Trained_models/trainednarrators/baseline/bart-base/"

In [3]:
# instantiate the narrator
narrator = ClassificationPerformanceNarration(model_base = "facebook/bart-base",
                                              model_base_dir= trained_model_path,
                                              class_labels = ['Low',
                                                              'High'],
                                              is_balance=True,
                                              modeltype='baseline',
                                              max_preamble_len=160,
                                              max_len_trg=185,
                                              length_penalty=3.6,
                                              beam_size=8,
                                              repetition_penalty=1.5,
                                              return_top_beams=1,
                                              random_state=453,
                                              )
                                              
# Set up the inference model
narrator.buildModel()

In [4]:
# The setup is ready to be used

# Sample of the expected format for the performance report
performance_report = {'Recall':["89.56%","HIGH"],'Accuracy':["40.16%","LOW"],'AUC':["70.16%","HIGH"]}
generatedTexts = narrator.generateTextualExplanation(performance_report)

Global seed set to 453


In [5]:
generatedTexts

"On this multi-class classification task, where the test instances are classified under either class Low or class High, the model's performance is summarized by the scores: (a) Accuracy equal to 40.16%; (b) AUC score equal 89.56%. (c) Precision score equals 70.32%. Since there is a disproportionate amount of data between the two class labels, only the precision and Recall scores are important metrics to take into account. From these scores, we can draw the conclusion that this model will likely misclassify only a small portion of all possible test cases."

In [18]:
from transformers import BartPretrainedModel,BartConfig

In [25]:
narrator.narrator.model.generator.config.to_json_file(json_file_path=trained_model_path+'/config.json')

In [30]:
params_dict = json.load(open(trained_model_path+'/parameters.json'))
#state_dict = json.load(open(args.model_base_dir+'/parameters.json'))
best_check_point = params_dict['best_check_point']
best_check_point_model = best_check_point + '/pytorch_model.bin'
state_dict = torch.load(best_check_point_model)

In [26]:
bconfig = BartConfig.from_json_file(trained_model_path+'/config.json')

In [27]:
zzp = BartPretrainedModel(bconfig)

In [31]:
zzp.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for BartPretrainedModel:
	Unexpected key(s) in state_dict: "generator.final_logits_bias", "generator.model.shared.weight", "generator.model.encoder.embed_tokens.weight", "generator.model.encoder.embed_positions.weight", "generator.model.encoder.layers.0.self_attn.k_proj.weight", "generator.model.encoder.layers.0.self_attn.k_proj.bias", "generator.model.encoder.layers.0.self_attn.v_proj.weight", "generator.model.encoder.layers.0.self_attn.v_proj.bias", "generator.model.encoder.layers.0.self_attn.q_proj.weight", "generator.model.encoder.layers.0.self_attn.q_proj.bias", "generator.model.encoder.layers.0.self_attn.out_proj.weight", "generator.model.encoder.layers.0.self_attn.out_proj.bias", "generator.model.encoder.layers.0.self_attn_layer_norm.weight", "generator.model.encoder.layers.0.self_attn_layer_norm.bias", "generator.model.encoder.layers.0.fc1.weight", "generator.model.encoder.layers.0.fc1.bias", "generator.model.encoder.layers.0.fc2.wei